# Lab 3: Visualizations

Please complete this lab by providing answers in cells after the question. Use **Code** cells to write and run any code you need to answer the question and **Markdown** cells to write out answers in words. After you are finished with the assignment, remember to download it as an **HTML file** and submit it in **ELMS**.

This assignment is due by **11:59pm on Friday, September 23**.

In [ ]:
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots

## Reviewing Table Operations

Let's review some of the table operations we went over last week. We used methods such as `group` and `pivot` to get summaries of categorical variables. Let's take a look again at how it works using the `imdb` and `population` datasets.

In [ ]:
imdb = Table.read_table('imdb.csv')
imdb.show(5)

In [ ]:
population_amounts = Table.read_table("world_population.csv")
years = np.arange(1950, 2016)
population = population_amounts.with_columns("Year", years)
population.show(5)

### `group`
The table method `group` takes as its argument a string representing a column name. This gives the count of each category of that variable. **Remember to only use this with categorical variables, since it won't really make sense with numerical data.** Note that even if a variable has numbers, that doesn't mean it's a numerical variable. For example, we can find the number of movies that were released in each decade using `group` with the `Decade` variable.

In [ ]:
movies_by_decade = imdb.group('Decade')
movies_by_decade

We can also use the `collect` parameter to summarize other variables split up by the categories in `Decade`.

In [ ]:
imdb.group('Decade', collect = np.mean)

### `pivot`

You can use `pivot` to create contingency tables, looking at the counts for multiple categorical variables. You can also use `pivot` to find, for example, the mean of a third variable within each combination of categories. We'll focus on the first one for now. Let's say we want find out what the distribution of movies that got rated higher than 8.2 was by decade.

We first need to create a column that determines whether a movie had a rating above 8.2. We can do this with the following code.

In [ ]:
# Just run this cell

imdb = imdb.with_column('above_8_2', imdb.column('Rating') > 8.2)

In [ ]:
imdb.show(5)

In [ ]:
imdb.pivot('above_8_2','Decade')

In [ ]:
imdb.pivot('above_8_2','Decade', values = 'Rating', collect = np.mean)

<font color = 'red'>**Question 1. Suppose we wanted to find the median rating of movies by decade. How might we do this?**</font>

*Hint:* You might want to use `np.median`.

<font color = 'red'>**Question 2. What if we wanted to figure out how many movies had a rating greater than 8.2 and more than 50 years old. How might we do this?**</font>

## Visualizations

Why did we spend so much time on manipulating data? Because to get the visualization that we want, we're going to need to use many of these tools. In this section, we'll use the `imdb` dataset to explore some basic descriptive statistics and data visualizations.

### Histograms and Boxplots

We can make histograms by using the `.hist` method. To do this, though, you need to make sure that you only include the variable that you want as a Table. That is, you need to make sure that your data is in this form:

In [ ]:
imdb_rating = imdb.select('Rating')
imdb_rating.show(5)

We then use the `.hist` method to create a histogram.

In [ ]:
imdb_rating.hist()

You can control characteristics about the histogram using parameters such as `bins`.

In [ ]:
imdb_rating.hist(bins = 5)

### Boxplots

Another visualization you can use for numerical data is a boxplot. As you might be able to guess, you can create this using the `boxplot` method. The data must be in a similar format.

In [ ]:
imdb_rating.boxplot()

<font color = 'red'>**Question 3. Make a histogram of the number of votes that movies got. What can you say about the shape and center of the distribution based on the histogram? Try using a few different bin values. Which one would be best to use?**</font>

<font color = 'red'>**Question 4. Make a boxplot of the number of votes that movies got. What can you say about the shape and center of the distribution?**</font>

### Line Plot

Let's say we want to look at the change in movie ratings over time. To investigate this, we'll plot the mean rating over time. 

First, we need to get the data in the form we want it. We want a column with the different decades in our dataset (the x-axis) as well as a column with the mean rating for that decade (the y-axis). To obtain a table with this data, we can use the `group` method, using `collect = np.mean` as a parameter.

In [ ]:
rating_by_decade = imdb.select('Decade','Rating').group('Decade', collect = np.mean)
rating_by_decade

In [ ]:
# The variable to go on the x-axis is specified as a parameter
rating_by_decade.plot('Decade')

<font color = 'red'>**Question 5. Make a line plot of the world population over time. What is the overall trend in population?**</font>

## Categorical Variables: Bar Chart

To make bar charts, we first need to create the summaries of the groups that we want to graph. Remember: bar charts are used for categorical variables. This means that we want to get the **counts** for each **category** in that variable. We can do this using the `group` method. 

In [ ]:
movies_by_decade = imdb.group('Decade')
movies_by_decade.show(4)

We take this table and use the `barh` method, specifying which variable we want to create the graph of.

In [ ]:
movies_by_decade.barh('Decade')

To create a bar chart with multiple categories, we can use the `pivot` method to create a contingency table and use that instead of the `group` method.

In [ ]:
imdb.pivot('above_8_2','Decade').barh('Decade')

<font color = 'red'>**Question 6. Create a bar graph of the number of movies that had a rating above 8 or below 8. Based on this graph and the graph of movies split by rating by decade, does any decade look particularly unusual to you?**</font>

## Joins

Sometimes, you might want to use data from multiple tables to analysis or create visualizations. We've already worked with creating Tables using existing arrays like when we added `Year` to the `population` Table. But sometimes, the rows don't quite match up exactly, so we might need to use `join`s to create a new Table that combines two Tables based on a common variable.

For example, let's say you wanted to know the world population in the year that each movie came out. The `imdb` table has information about movies and the year they came out in, while the `population` table has the world population for each year. We can `join` these two tables to combine the data.

To do this, we will use format `<table 1>.join(<table 1 column for joining>, <table 2>, <table 2 column for joining>)`.

In [ ]:
movie_population = imdb.join('Year', population, 'Year')
movie_population.show(5)

Note that the column for joining is the variable that is the same across the two Tables. The `join` method will combine all the columns from both datasets and match up the rows based on which values are the same in the variables specified in the first and third arguments.

So, in this case, we have a `Year` column in the `imdb` Table, and a `Year` column in the `population` Table. The first argument is indicating that the `Year` in the `imdb` Table should be used to match. The third argument is indicating that the `Year` in `population` should be used to match. 

### `relabel`

We can use `relabel` to change the names of columns in the Table. Let's change the name of the `Population` column that we just joined in to make it more descriptive.

In [ ]:
movie_population.relabel('Population', 'World Pop. When Movie Was Released')

<font color = 'red'>**Question 7. What if we wanted to know the world population at the start of the decade that the movie came out in? Make a new Table by adding a column to `movie_population` that contains the population at the start of that decade. Change the name of that column to `Population at Start of Decade`. Call the new table `movie_population_with_decade`.**</font>

*Hint:* Think about what variables there are already within the `movie_population` Table. 